In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [35]:
class Stateless_RNN:
    def __init__(self,n_steps = 200):
        self.n_steps = n_steps
        self.model = None
        self.dataset = None        
        self.max_id = None
        self.tokenizer = None 
    
    def prepare_data(self,path, batch_size = 39):
        with open(path ,encoding="utf8") as f:
            text=f.read()
            
        self.tokenizer = keras.preprocessing.text.Tokenizer(char_level = True)
        self.tokenizer.fit_on_texts([text])
        self.max_id = len(self.tokenizer.word_index)
    
        [encoded] = np.array(self.tokenizer.texts_to_sequences([text])) -1
        train_size = len(encoded) * 90//100
        
        self.dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
        
        
        window_length = self.n_steps + 1
        self.dataset = self.dataset.window(window_length , shift =1 ,drop_remainder = True)
        self.dataset = self.dataset.flat_map(lambda window : window.batch(window_length))

        
        self.dataset = self.dataset.shuffle(10000).batch(batch_size)
        self.dataset = self.dataset.map(lambda windows:(windows[:,:-1] , windows[:,1:]))
        self.dataset = self.dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=self.max_id),Y_batch))
        self.dataset = self.dataset.prefetch(1)
        
    def build_layers(self):
        self.model = keras.models.Sequential([
        keras.layers.GRU(128, return_sequences=True, input_shape=[None,self.max_id], dropout=0.2, recurrent_dropout=0.2),
        keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        keras.layers.TimeDistributed(keras.layers.Dense(self.max_id,activation="softmax"))])
        
        return self.model.summary()

    def train(self,epochs=5):
        self.model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
        self.model.fit(self.dataset, epochs=epochs)
    
    def save_model(self,path):
        self.model.save(path)
        
    def load_model(self,path):
        self.model = keras.models.load_model(path)

    
    def preprocess_input(self,text):
        X = np.array(self.tokenizer.texts_to_sequences(text)) -1
        return tf.one_hot(X,depth =self.max_id)
    
    def predict_next_char(self,text,temperature):
        X_new = self.preprocess_input([text])
    
        y_proba = self.model.predict(X_new)[0, -1:, :]

        rescaled_logits = tf.math.log(y_proba) / temperature

        char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
        return self.tokenizer.sequences_to_texts(char_id.numpy())[0]
    
    def complete_text(self,text, n_chars=200, temperature=1):
        for _ in range(n_chars):
            text += self.predict_next_char(text, temperature)
        return text


    

In [36]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()


In [62]:
class Statefull_RNN:
    def __init__(self,n_steps = 200):
        self.n_steps = n_steps
        self.batch_size = None
        self.model = None
        self.dataset = None        
        self.max_id = None
        self.tokenizer = None
    
    def prepare_data(self,path, batch_size = 1):
        self.batch_size = batch_size
        with open(path ,encoding="utf8") as f:
            text=f.read()
            
        self.tokenizer = keras.preprocessing.text.Tokenizer(char_level = True)
        self.tokenizer.fit_on_texts([text])
        self.max_id = len(self.tokenizer.word_index)
    
        [encoded] = np.array(self.tokenizer.texts_to_sequences([text])) -1
        train_size = len(encoded) * 90//100
                       
        
        window_length = self.n_steps + 1
        dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
        dataset = dataset.window(window_length, shift=self.n_steps,
        drop_remainder=True)
        dataset = dataset.flat_map(lambda window: window.batch(window_length))
        dataset = dataset.batch(1)
        dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
        dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=self.max_id), Y_batch))
        dataset= dataset.prefetch(1)
        self.dataset = dataset
        
    def build_layers(self):
        self.model = keras.models.Sequential([keras.layers.GRU(128, return_sequences=True, stateful=True,
        dropout=0.2, recurrent_dropout=0.2,batch_input_shape=[self.batch_size, None, self.max_id]),
        keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, recurrent_dropout=0.2),
        keras.layers.TimeDistributed(keras.layers.Dense(self.max_id, activation="softmax"))
        ])

    def train(self,epochs=5):
        self.model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
        self.model.fit(self.dataset, epochs=50, callbacks=[ResetStatesCallback()])

    
    def save_model(self,path):
        self.model.save(path)
        
    def load_model(self,path):
        self.model = keras.models.load_model(path)

    
    def preprocess_input(self,text):
        X = np.array(self.tokenizer.texts_to_sequences(text)) -1
        return tf.one_hot(X,depth =self.max_id)
    
    def predict_next_char(self,text,temperature):
        X_new = self.preprocess_input([text])
    
        y_proba = self.model.predict(X_new)[0, -1:, :]

        rescaled_logits = tf.math.log(y_proba) / temperature

        char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
        return self.tokenizer.sequences_to_texts(char_id.numpy())[0]
    
    def complete_text(self,text, n_chars=200, temperature=1):
        for _ in range(n_chars):
            text += self.predict_next_char(text, temperature)
        return text


    

In [67]:
model = stateless_rnn_model()
model.prepare_data('hafez1.txt')
model.build_layers()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_20 (GRU)                 (None, None, 128)         68736     
_________________________________________________________________
gru_21 (GRU)                 (None, None, 128)         99072     
_________________________________________________________________
time_distributed_10 (TimeDis (None, None, 49)          6321      
Total params: 174,129
Trainable params: 174,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.train(epochs=25)

In [79]:
print(model.complete_text(text ='ساقی',n_chars=100))

ساقی چشمت بين و من جان بينم

به صوت ملک هلای توزو می‌بينم

لب در سر حسن و روزی تو شد از هم

گر نتورم دری


In [81]:
print(model.complete_text(text ='من',n_chars=100))

منزل ريا که روز و بيشت در زلف تو در سرم

گر به آب جورش علم گشا از پادشهان
چند نظمی آگه می‌زنمت خبر درد


In [82]:
print(model.complete_text(text ='خانه',n_chars=100))

خانه را
چرخ عشق و من از ياد که بارسان دردم

شهبازی نه چندان نيست و دل غيرت می‌کنم

پير مغان ببينم و می د


In [78]:
print(model.complete_text(text ='دوش',n_chars=700))

دوش لطف عشق می‌رود

ای اميدم ز قراراتم در آن خشت و خوش

من خاک سرو گفت که در جان دردی بازم

دوش ماه چشم گر غمی ز صد وفا آوردم


غزل    ۳۶۱

آتش دلبر يک دو صد هر روی ره نگارم

چو صباح می‌بينی و کفن عارض است
همنم فيضی توست نيست پاکدايم

چال عرش نه اين حريفان نبرد هم

گر به شوق زان عذوه‌ای ز کار هم

آن کو می‌نام جوی روحی چه کنم

اليست چرا مذهب را که هم

روز عالم نام و طلب ما هست
مرا
عاشقان آمده‌ای روحی تو نس
يدادن چه عذر ناتوشم بينی بنشينم


غزل    ۳۵۷

روزگاری و مرا چون پرور عهد بشود آن ماه
همرايت که با دلدام جان و علم زدنيست
طربسار برخيزيم و کان در نظر ليل

حافظ از ياد روان با آمده ما بسی تا می‌بينم

هرگز و نظر باده از آن خاک گو می‌بينم

با دل از پی آزادگان می‌کنم

عرزاست که می‌گذرم هر دم به طوف


In [63]:
stateful_model =Statefull_RNN()
stateful_model.prepare_data('hafez1.txt')
stateful_model.build_layers()


In [ ]:
stateful_model.train(epochs=1)

In [66]:
print(stateful_model.complete_text('ساقی'))

ساقی آن بهر حايت

لغال  ع۲۶۲۰

تا صب گران ايتوا سرحاسن قا ته خاسان

سخظ حافظتمم لوی مارش کنيگوش

انگ چه عاهر منيش تو بهته ما حرتان
بخانيای شاقی خوار مخر ميوی
صببه که می‌شدد و خل
و دمد دود شاشافيا مض
ديار 
